In [ ]:
Random Forest
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as srn
import plotly
import pickle
from sklearn.ensemble import RandomForestClassifier
from yellowbrick.classifier import ConfusionMatrix
from sklearn.metrics import classification_report

# Base Credit Data

## Pré-processamento

In [ ]:
with open('credit.pkl', 'rb') as f:
    X_credit_train, y_credit_train, X_credit_test, y_credit_test = pickle.load(f)

In [ ]:
print(X_credit_train.shape, y_credit_train.shape)
print(X_credit_test.shape, y_credit_test.shape)

# Base Census

## Pré-processamento